# Project: Customer Churn Prediction System

### Introduction
Customer churn, the rate at which customers stop doing business with a company, is a critical metric for any subscription-based service. Acquiring a new customer can be five to 25 times more expensive than retaining an existing one. Therefore, proactively identifying customers who are at a high risk of churning is essential for sustainable business growth.

This notebook details the development of a machine learning system to predict customer churn for a telecommunications company. We will leverage a historical dataset of customer interactions and attributes to train a model that can forecast the likelihood of a customer leaving.

### Problem Statement
The business objective is to reduce customer churn by identifying at-risk customers *before* they leave. By predicting churn probability, the company can deploy targeted retention strategies, such as offering special discounts, providing proactive customer support, or upgrading services. This data-driven approach aims to improve customer satisfaction and increase the company's revenue and market share.

### Project Objectives

1. **Exploratory Data Analysis (EDA):** Understand the dataset and identify the key factors that correlate with customer churn.  
2. **Data Preprocessing & Feature Engineering:** Prepare the data for machine learning by handling missing values, encoding categorical variables, and scaling numerical features.  
3. **Model Training:** Build and train several classification models (Logistic Regression, Random Forest, and XGBoost) to predict churn.  
4. **Model Evaluation:** Assess the performance of the models using metrics like accuracy, precision, recall, F1-score, and the ROC-AUC curve. Select the best-performing model.  
5. **Insight Generation:** Analyze the feature importances from our best model to understand the primary drivers of churn and provide actionable business recommendations.  


### Step 1: Setup and Data Loading

First, we'll import all the necessary Python libraries for data manipulation, visualization, and machine learning. We will then load the Telco Customer Churn dataset into a pandas DataFrame.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report, roc_curve

# Set visualization style
sns.set_style('whitegrid')


/home/charles/anaconda3/envs/learn-env/lib/python3.8/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [2]:
# Load the dataset
df = pd.read_csv('../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### Step 2: Initial Data Exploration and Cleaning
We check its structure, data types, and look for any missing values.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


**Observations:**  
- We have 7043 customer records and 21 columns.  
- `customerID` is a unique identifier and can be dropped.  
- `SeniorCitizen` is an integer but acts as a categorical feature (0 or 1).  
- `TotalCharges` is an `object` type (string), but it should be numeric. This suggests there might be non-numeric values in it.  

Let's fix `TotalCharges`.  


In [4]:
# Convert 'TotalCharges' to numeric, coercing errors to NaN (Not a Number)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Check for missing values again
print(df.isnull().sum())


customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64


We see that 11 rows now have missing `TotalCharges`. These likely correspond to new customers with zero tenure who haven't been billed yet. Since it's a very small number of rows, we can impute them. We use the median, which is robust to outliers.

In [5]:
# Impute missing values with the median
median_total_charges = df['TotalCharges'].median()
df['TotalCharges'].fillna(median_total_charges, inplace=True)

# Drop the customerID column as it's not a predictive feature
df.drop('customerID', axis=1, inplace=True)

# Convert our target variable 'Churn' into a binary format (0 or 1)
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
